# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

CLASSIFICATION
This is a classification problem as we have a discrete-valued output. (1 or 0, meaning "passed" or "not passed" in our case).
In fact a regression would predict continuous valued output, instead.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [63]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = (student_data['passed']=='yes').sum()

# TODO: Calculate failing students
n_failed = (student_data['passed']=='no').sum()

# TODO: Calculate graduation rate
grad_rate = float (n_passed) / (n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [62]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [64]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [89]:
# TODO: Import any additional functionality you may need here
import numpy as np
from sklearn.cross_validation import train_test_split



# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, train_size = num_train, random_state=42)

# X_train = None
# X_test = None
# y_train = None
# y_test = None

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

Before proceeding with the explanation of the details for the chosen models, I'd like to spend few words on the ones that I haven't selected.
- LOGISTIC REGRESSION: I haven't chosen this model as it is more suitable for linear classifications, unless we modify the features.
- STOCHASTIC GRADIENT DESCENT: This model is suitable for regression problems, therefore I haven't selected it.
- NAIVE-BAYSES: The main reason why I have't used it is beacause it assumes independent predictors, meaning that all properties contribute independently to the probability. In our problem this might not be the case. Furthermore if a categorical variable has a category in the test dataset which was not in the training dataset, the model assigns zero probability and therefore it is unable to predict. Of course there would be solution to that, such as Laplace smoothing technique. In our case, though, we could encounter such a scenario.
- DECISION TREE: I honestly was debating the use of this model until I made a final decision to use the ENSEMBLE method instead. The main reason why I haven't chosen D.T. is beacause it's prone to overfitting , especially with data with a lot of features. A solution would be to stop the growth of a tree, but for this reason I've chosen the "Random forests" ENSEMBLE method that can overcome the issue of high variance and built a classifier on top of a classifier, washing out the trees that query on useless features.

Having said that, here below are the models that I have chosen and their characteristics.

-SUPPORT VECTOR MACHINES (SVM): 
    STRENGTHS: 
      - It works well in complicated domains 
      - It works good with outliers (doesn't get affected by them as it uses more relevant points to make the 
        separation)
      - It works for non-linear classification (using kernel, such as Gaussian Kernels)
    WEAKNESSES:
      - Doesn't work well for large dataset and with a lot of noise
      - The choice of a good Kernel might be complicated
      - It does not provide probability estimate (which in our case is not neeed)
      - Poor performance if the number of features is higher than the number of training samples
    WHY A GOOD CANDIDATE FOR THE PROJECT?
      Not knowing how the data are positioned (if they are linearly separable or not) I thought about using this model
      as it works well for non-linear separation and with complicated domains.
      Also, being the dataset of a fairly small size, I don't have to worry about the weakness of the model in dealing
      with large training set.
      Also, not knowing how the data are positioned I don't have to worry about a model that might be affected by 
      outliers.
    REAL-WORLD APPLICATION:
      I have also found several reference in literature, such as this book: 
      http://link.springer.com/article/10.1007/s10462-010-9166-x 
      where it is outlined that SVM has made a large contribution to the development of spam email filtering. The
      book goes in detail in describing the different kernels used, such as distance-based kernels. Finally it 
      proposes a different approach throught the use of string kernels for spam filtering.
      Furthermore, I have taken the Andrew Ng class and one of the project was the use of SVM for spam detection.
      I was able to test myself the performance of the model and it actually gave very good results:
      99.8% training accuracy and 98.5% test accuracy.
      
-K-NEAREST NEIGHBORS (KNN):
    STRENGTHS: 
      - Robust to noisy training data
      - It can detect linear or non-linear distributed data
      - It is non-parametric, therefore it takes no assumption about the underlying data or its distribution
    WEAKNESSES:
      - Being non-parametric it is actually slow in querying (vs being fast in training)
      - It can be sensitive to outliers
    WHY A GOOD CANDIDATE FOR THE PROJECT?
      The main reason is the fact that it takes no assumption about the underlying data and its distribution. I 
      Thought that this would be good since I don't have much information about the data and I'd rather 
      be cautious and pick a model that can perform well without any assumption.
    REAL-WORLD APPLICATION:
      Finance is a field where this model have been used. One example that I found was in predicting whether the 
      price of a certain stock would increase of decrease, looking at past behaviour. 
      (ref. https://www.r-bloggers.com/using-knn-classifier-to-predict-whether-the-price-of-stock-will-increase/)
      
-ENSEMBLE METHOD (Random Forests)
    STRENGHTS:
      - It handles very well high dimensional spaces
      - It reduces high-variance (and this drove my choice for this model vs the Decision Tree)
      - It does not expect linear features or features that interact linearly
      - It improves the robustness of the classifier when used on decision trees. In fact, some trees query
      on useless features but others do not and instead make good predictions. So the random ones wash out as
      noise, while the good ones have a clear effect on the classification
    WEAKNESSES:
      - Lack of interpretation (a combination of decision trees is harder than a single tree)
      - It somehow violates the OCKHAM's RAZOR LAW, which implies that simplicity leads to greater accuracy.
    WHY A GOOD CANDIDATE FOR THE PROJECT?
      Once again, not knowing the distribution of the data and their interaction among each other, I wanted to
      use a model that is not affected by it. I liked the fact that is not affected by outliers and the feature 
      selection is sort of done automatically. The advantage of this model is also the fact that it overcomes 
      overfitting.
    REAL-WORLD APPLICATION:
      I was intrigued about how well this model performed in a competition from Netflix (Netflix Prize Competition)
      where the contestants were asked to predict user ratings for films, based on previous ratings without any other 
      information about the users or films (ref https://www.youtube.com/watch?v=coeak1YsaYc).
    

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [92]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [129]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = svm.SVC(random_state=10)
clf_B = KNeighborsClassifier()
clf_C = RandomForestClassifier(random_state=10)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0018 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained model in 0.0041 seconds
Made predictions in 0.0034 seconds.
F1 score for training set: 0.8679.
Made predictions in 0.0017 seconds.
F1 score for test set: 0.7815.
Training a SVC using a training set size of 300. . .
Trained model in 0.0085 seconds
Made predictions in 0.0072 seconds.
F1 score for training set: 0.8761.
Made predictions in 0.0018 seconds.
F1 score for test set: 0.7838.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.7246.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.00

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0092 seconds          | 0.0009 seconds         | 0.8777           | 0.7746          |
| 200               | 0.0050 seconds          | 0.0018 seconds         | 0.8679           | 0.7815          |
| 300               | 0.0076 seconds          | 0.0016 seconds         | 0.8761           | 0.7838          |

** Classifer 2 - KNeighborsClassifier  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0032 seconds          | 0.0012 seconds         | 0.8060           | 0.7246          |
| 200               | 0.0007 seconds          | 0.0015 seconds         | 0.8800           | 0.7692          |
| 300               | 0.0008 seconds          | 0.0021 seconds         | 0.8809           | 0.7801          |

** Classifer 3 - RandomForestClassifier  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0314 seconds          | 0.0009 seconds         | 0.9764           | 0.7538          |
| 200               | 0.0237 seconds          | 0.0009 seconds         | 0.9854           | 0.7463          |
| 300               | 0.0245 seconds          | 0.0010 seconds         | 0.9903           | 0.6917          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

The model that I choose to use is the SVM. Out of the three selected the first two (SVM and KNN) are outperforming the Ensemble method, when looking at the F1 score.
On the other hand SVM and KNN are performing very similarly, but I believe the first one has more robust peformance. 
In fact the F1 score is better in every training size, performing even better when there are smaller samples.
Prediction time, although slightly, is faster.
Training time, instead, is slower but I was expecting this one and I don't consider it a downside. In fact KNN is obviously faster in training time, being non-parametric, although the query time is slower. But the training time is something we can invest in at the beginning and take advantage instead of prediction time that can be isntead run more frequently.
Additionally if we care about space a parametric approach is the way to go as we dont need to store the original data.

In conclusion:
-based on the available data, a small training set, I would choose SVM as it performs better on smaller dataset
-if we have limited resources I would rather setup a model once and then run it faster for every query, without spending time instead of re-rnning the model for each new query
-cost goes as a consequence of the statement above. Faster query time and space efficency all count in terms of cost.
-performance is well explained by the F1 score which identifies the measure of test accuracy and in our case we want to be accurate in predicting which student will pass or not.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

The goal of a Support Vector Machine (SVM) is to analyze the given data and find the line that best separates, but committing the least, the existing data.
Obviously, when looking for a line that separates the data we can encounter several ones. The best line is the one that not only classifies the data in two classes, but also maximizes the margin (the distance to the nearest point) between the line itself and the data.
One characteristic of the SVM is its robustness in the sense that it “ignores” the outliers.

This looks pretty easy if we think about data spread across a 2-dimensional graph and somehow already “divided” into two different areas. It’s just a matter of finding the line that best maximizes the margin and that’s pretty much it.
A different story is instead when we have data that are scattered in different “shapes”, but this approach is valid 
for any other type of data positioned on the 2-dimensional space. Meaning, we can find a feature that can be used as an additional input, which can “transform” the data into linearly separable. This line, created in this new space, can then be “translated” back into the original space and provide a non-linear shape that separates the data.
This is called the “Kernel trick”. It basically finds functions that take a low dimensional input space and map it to a high dimensional space, so what it used to be not linearly separable now becomes as such.
Finally we can take the solution (the linear separation) from the high dimensional space to the low dimensional space and we deduct a non-linear separation.
The space is therefore devided into different areas separating the two classes (in our case "pass" and "no pass") and  a new data point will be categorized based on its position within the area it falls in.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [191]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV 
from sklearn.metrics import f1_score, make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear','rbf', 'poly'), 'C':[0.01,0.5,1]}

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0029 seconds.
Tuned model has a training F1 score of 0.8620.
Made predictions in 0.0011 seconds.
Tuned model has a testing F1 score of 0.7891.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

The F1 score for the tuned model improves in the test case, while decreases for training. C in fact controls the tradeoff between smooth decision boundary and classifying correctly. A low value of C in fact would get a smoother separator vs getting more training points correctly. This explains the behavior of F1, which is lower (than the untuned mdoel) for the training set, but higher the test set.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.